<a href="https://colab.research.google.com/github/nadia-maarfavi/YouTube.Data.Scraper/blob/main/YouTubeChanel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from googleapiclient.discovery import build
import time
import datetime

In [ ]:
youTubeApiKey="....." #Get YouTube API Key using this instruction: https://developers.google.com/youtube/v3/getting-started#
youtube=build('youtube','v3',developerKey=youTubeApiKey)
channelId='UCAjh5TggrX5Vj-FsoFeW3IA' #To obtain the channel id you can view the source code of the channel page and find https://www.youtube.com/channel/.....



In [ ]:
#Find the stat data of the chanel
statdata=youtube.channels().list(part='statistics',id=channelId).execute()
statdata

{'kind': 'youtube#channelListResponse',
 'etag': '05OoUR1GAQs3mvgRUe0dXgOL59M',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '4PebnzCb55GB4H8FuVBu0mKFw9A',
   'id': 'UCAjh5TggrX5Vj-FsoFeW3IA',
   'statistics': {'viewCount': '31780',
    'subscriberCount': '94',
    'hiddenSubscriberCount': False,
    'videoCount': '13'}}]}

In [ ]:
#To extract chanel's name
snippetdata=youtube.channels().list(part='snippet',id=channelId).execute()
title=snippetdata['items'][0]['snippet']['title']
title

'KIDS CRAFT ORIGAMI'

In [ ]:
description=snippetdata['items'][0]['snippet']['description']
description

'Craft Origami '

In [ ]:
# This code will extract the videos' ID in the chanel
contentdata=youtube.channels().list(id=channelId,part='contentDetails').execute()
playlist_id = contentdata['items'][0]['contentDetails']['relatedPlaylists']['uploads']
videos = []
next_page_token = None

while 1:
    res = youtube.playlistItems().list(playlistId=playlist_id,
                                    part='snippet',
                                    maxResults=50,
                                    pageToken=next_page_token).execute()
    videos += res['items']
    next_page_token = res.get('nextPageToken')

    if next_page_token is None:
        break

In [ ]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))
video_ids

['4SpXCwFhxnc',
 'EhK5U2RKfmA',
 'yVFp3xIFNoA',
 'Ar0CBiT3UEU',
 'LMIjerNT5EQ',
 'EO6yANViv5k',
 'Mw4dkqC3C3o',
 '_08J2mohXLk',
 'ppsxwwyaZcI',
 'n0IDL3s1rDc',
 'PlIytYZrKq0',
 '0VDor7hOpEg',
 'UlQ7tZW7xcs']

In [ ]:
# This code extract the information about each video such as number of likes, comments and ect
stats = []
for i in range(0, len(video_ids), 40):
    res = (youtube).videos().list(id=','.join(video_ids[i:i+40]),part='statistics,contentDetails').execute()
    stats += res['items']

In [ ]:
# The inf
stats[0]

{'kind': 'youtube#video',
 'etag': 'AyabTkjDouOyr29qN9QE2OADd2w',
 'id': '4SpXCwFhxnc',
 'contentDetails': {'duration': 'PT5M56S',
  'dimension': '2d',
  'definition': 'hd',
  'caption': 'false',
  'licensedContent': False,
  'contentRating': {},
  'projection': 'rectangular'},
 'statistics': {'viewCount': '67',
  'likeCount': '2',
  'favoriteCount': '0',
  'commentCount': '0'}}

In [ ]:
print(len(stats))

13


In [ ]:
# create a dataframe containts all the information about the videos in the chanel
title=[]
liked=[]
disliked=[]
views=[]
favorite = []
comment=[]
PublishDate = []
Date = []
Time = []
DateTimeFull = []
Duration = []
for i in range(len(videos)):
        title.append((videos[i])['snippet']['title'])
        date = ((videos[i])['snippet']['publishedAt'])
        Duration.append(stats[i]['contentDetails']['duration'])
        date_time_obj = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")
        PublishDate.append((videos[i])['snippet']['publishedAt'])
        Date.append(date_time_obj.date())
        Time.append(date_time_obj.time())
        DateTimeFull.append(date_time_obj)

        views.append(int((stats[i])['statistics']['viewCount']))
        try:
          liked.append(int((stats[i])['statistics']['likeCount']))
        except KeyError as l:
          liked.append(0)

        try:
          disliked.append(int((stats[i])['statistics']['dislikeCount']))
        except KeyError as l:
          disliked.append(0)

        try:
          favorite.append(stats[i]['statistics']['favoriteCount'])
        except KeyError as l:
          favorite.append(0)

        try:
          comment.append(int((stats[i])['statistics']['commentCount']))
        except KeyError as l:
          comment.append(0)


In [ ]:
import pandas as pd
data={'title':title,'Duration':Duration,'Published Date':PublishDate,'Views':views,'liked':liked, 'Disliked':disliked, 'Favourite':favorite, 'Comments':comment,'Date':Date,'Time':Time,'FullDateTime':DateTimeFull}
df=pd.DataFrame(data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           13 non-null     object        
 1   Duration        13 non-null     object        
 2   Published Date  13 non-null     object        
 3   Views           13 non-null     int64         
 4   liked           13 non-null     int64         
 5   Disliked        13 non-null     int64         
 6   Favourite       13 non-null     object        
 7   Comments        13 non-null     int64         
 8   Date            13 non-null     object        
 9   Time            13 non-null     object        
 10  FullDateTime    13 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 1.2+ KB


In [ ]:
df

,title,Duration,Published Date,Views,liked,Disliked,Favourite,Comments,Date,Time,FullDateTime
0,How To Make A Paper Bat Origami. Halloween Dec...,PT5M56S,2023-10-30T18:44:36Z,67,2,0,0,0,2023-10-30,18:44:36,2023-10-30 18:44:36
1,Paper Spide Web! Halloween Decorations. Happy ...,PT7M17S,2023-10-22T11:25:18Z,38,2,0,0,1,2023-10-22,11:25:18,2023-10-22 11:25:18
2,How to Make an Easy Ghost Origami _ Happy Hall...,PT5M2S,2023-10-17T16:52:34Z,45,2,0,0,0,2023-10-17,16:52:34,2023-10-17 16:52:34
3,How To Make a Paper Spider Web _ Halloween Dec...,PT5M54S,2023-10-17T12:24:16Z,133,3,0,0,0,2023-10-17,12:24:16,2023-10-17 12:24:16
4,How to Make a Three Wings Boomerang!!!,PT5M50S,2023-09-01T16:22:20Z,1286,10,0,0,1,2023-09-01,16:22:20,2023-09-01 16:22:20
5,How to Make a Boomerang - Let's Throw it!,PT7M47S,2023-08-29T20:12:49Z,642,3,0,0,1,2023-08-29,20:12:49,2023-08-29 20:12:49
6,How to Make a Paper Plane Fly Long and Smooth!,PT6M42S,2023-08-24T15:27:20Z,15942,65,0,0,4,2023-08-24,15:27:20,2023-08-24 15:27:20
7,Creating a Concord Paper Airplane,PT5M32S,2023-08-22T23:23:17Z,1340,6,0,0,1,2023-08-22,23:23:17,2023-08-22 23:23:17
8,Easy DRAGON Origami _ Let's Fold,PT15M4S,2023-08-16T23:17:49Z,268,5,0,0,0,2023-08-16,23:17:49,2023-08-16 23:17:49
9,How To Make an Easy DINOSAUR _ Paper Dino,PT6M18S,2023-08-16T21:09:54Z,73,4,0,0,0,2023-08-16,21:09:54,2023-08-16 21:09:54
